In [ ]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt


import gc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# pd.set_option('display.max_columns',None)

In [ ]:
%%time
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
df = df.astype(np.float16)
# df.drop(['row_id','time_id'],axis=1,inplace=True)
print(df.shape)
df.head()

In [ ]:
# # target correlation
# corr_data = df[df.keys()]
# cmap = plt.cm.PuBu
# cols = corr_data.corr().nlargest(50,'target')['target'].index
# cm = np.correof(df[cols].values.T)
# f,ax = plt.subplots(figsize=(25,15))
# heatmap = sns.heatmap(cm,vmax=1,linewidths=0.1,square=True,annot=True,cmap=cmap,linecolor='white',
#                      xticklabels=cols.values,yticklabels=cols.values)

In [ ]:
investment_id = df.pop('investment_id')
investment_id.head()

In [ ]:
target = df.pop('target')
target.head()

In [ ]:
train.info()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=1)
train_pca = pca.fit_transform(train)

In [ ]:
train

In [ ]:
print(train.shape,target.shape)

In [ ]:
from lightgbm import LGBMRegressor
from lightgbm import plot_importance,plot_metric
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [ ]:
del df
gc.collect()

In [ ]:
kf = KFold(n_splits=10,shuffle=True)
LGB_models = []
fold = 1
for (train_idx,valid_idx) in kf.split(train,target):
    print(f'Fold:{fold}')
    x_train, x_valid = train.values[train_idx], train.values[valid_idx]
    y_train, y_valid = target[train_idx], target[valid_idx]
    
    model = LGBMRegressor(objective='regression',n_estimators=10000,learning_rate=0.05,
                         max_depth=300,n_jobs=-1,device='gpu',subsample=0.7)
    model.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],
             eval_metric=['rmse'],early_stopping_rounds=100,verbose=100
             )
    LGB_models.append(model)
    fold += 1

In [ ]:
plot_metric(model)

In [ ]:
def ensemble(models):
    LGB_ensemble = []
    for model in models:
        pred = model.predict(test_df)
        LGB_ensemble.append(pred)
    return np.mean(LGB_ensemble,axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id'], axis=1, inplace=True)
    pred = ensemble(LGB_models)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df)